In [ ]:
import os
import sys
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
from copy import deepcopy
from matplotlib import pyplot as plt
from IPython.display import clear_output
from matplotlib.colors import TABLEAU_COLORS

sys.path.append('../../..')
from uncertain.utils.data import Data
from uncertain.utils.training import train, run_study, load
from uncertain.utils.evaluation import test_vanilla, test_uncertain, test_chap_five

from uncertain.implicit.base import MF, MLP
from uncertain.implicit.DoubleMF import JointDoubleMF, SequentialDoubleMF
from uncertain.implicit.extras import HeuristicUncertainty
from uncertain.implicit.neural import GaussianMLP, MCDropout, Ensemble

if os.path.isfile('data.pkl'):
    with open('data.pkl', 'rb') as f:
        data = pickle.load(f)
    print(f'Data prepared: {data.n_user} users, {data.n_item} items.')
    print(f'{len(data.train)} train, {len(data.val)} validation and {len(data.test)} test interactions.')
    data.batch_size = int(1e3)
else:
    data = pd.read_table('data.csv', sep=',', header=0)
    data.columns = ['user', 'item', 'score', 'timestamps']
    # data = data.drop(columns='timestamps') # Trying randomly shuffled data
    data = Data(data, implicit=True)
    with open('data.pkl', 'wb') as f:
        pickle.dump(data, f, protocol=5)

data.item_support = data.item_support.astype(float)
data.user_support = data.user_support.astype(float)        

base_batch_size = 1024 *5
trials = 0 ## 0 for eval only mode
patience = 2 ## Number of validation checks before ending training

In [ ]:
# Calculate the tail items accordind to AUR paper
import torch

n_inter = data.item_support.sum()
order = np.argsort(data.item_support)

for n_tail in range(data.n_item):
    ratio_tail = np.sum(data.item_support[order[:n_tail]]) / n_inter
    if ratio_tail > 0.5:
        print(n_tail)
        break

data.tail_items = torch.tensor(order[:n_tail])
data.head_items = torch.tensor(order[n_tail:])

# Baselines

## MF

In [ ]:
name = 'MF-BCE'
def init_model(**kwargs):
    return MF(data.n_user, data.n_item, embedding_dim=128, loss='BCE', **kwargs)

def objective(trial):
    
    # Parameter setup
    params = {'lr': trial.suggest_float('lr', 1e-4, 1e-2, log=True),
              'weight_decay': trial.suggest_float('wd', 1e-5, 1e-2, log=True),
              'n_negatives': trial.suggest_int('neg', 1, 50)}
    params_str = '-'.join(f'{key}={value}' for key, value in params.items())
    data.batch_size = int(base_batch_size / (params['n_negatives'] + 1))
    print(params, data.batch_size)

    # Train
    model = init_model(**params)
    MAP, path, train_likelihood = train(model, data, path='checkpoints/' + name, name=params_str, patience=patience)
    trial.set_user_attr('filename', path)
    trial.set_user_attr('mean_loss', train_likelihood)
    return MAP

study = run_study(name, objective, n_trials=trials)
best_runs = study.trials_dataframe().sort_values('value')[::-1][:5]
mfbce = init_model().load_from_checkpoint(best_runs.user_attrs_filename.iloc[0])

results = test_vanilla(mfbce, data, max_k=10, name=name)
clear_output(wait=True)
print(results['MAP'])
best_runs

In [ ]:
name = 'MF-BPR'
def init_model(**kwargs):
    return MF(data.n_user, data.n_item, embedding_dim=128, loss='BPR', **kwargs)

def objective(trial):
    
    # Parameter setup
    params = {'lr': trial.suggest_float('lr', 1e-4, 1e-2, log=True),
              'weight_decay': trial.suggest_float('wd', 1e-5, 1e-2, log=True)}
    params_str = '-'.join(f'{key}={value}' for key, value in params.items())
    data.batch_size = int(base_batch_size / 2)
    print(params, data.batch_size)

    # Train
    model = init_model(**params)
    MAP, path, train_likelihood = train(model, data, path='checkpoints/' + name, name=params_str, patience=patience)
    trial.set_user_attr('filename', path)
    trial.set_user_attr('mean_loss', train_likelihood)
    return MAP

study = run_study(name, objective, n_trials=0)
best_runs = study.trials_dataframe().sort_values('value')[::-1]
mfbpr = init_model().load_from_checkpoint(best_runs.user_attrs_filename.iloc[0])

results = test_vanilla(mfbpr, data, max_k=10, name=name)
# clear_output(wait=True)
print(results['MAP'])
best_runs

In [ ]:
results

In [ ]:
name = 'MF-MSE'
def init_model(**kwargs):
    return MF(data.n_user, data.n_item, embedding_dim=128, loss='MSE', **kwargs)

def objective(trial):
    
    # Parameter setup
    params = {'lr': trial.suggest_float('lr', 1e-4, 1e-2, log=True),
              'weight_decay': trial.suggest_float('wd', 1e-5, 1e-2, log=True),
              'n_negatives': trial.suggest_int('neg', 1, 50)}
    params_str = '-'.join(f'{key}={value}' for key, value in params.items())
    data.batch_size = int(base_batch_size / (params['n_negatives'] + 1))
    print(params, data.batch_size)

    # Train
    model = init_model(**params)
    MAP, path, train_likelihood = train(model, data, path='checkpoints/' + name, name=params_str, patience=patience)
    trial.set_user_attr('filename', path)
    trial.set_user_attr('mean_loss', train_likelihood)
    return MAP

study = run_study(name, objective, n_trials=trials)
best_runs = study.trials_dataframe().sort_values('value')[::-1]
mfmse = init_model().load_from_checkpoint(best_runs.user_attrs_filename.iloc[0])

results = test_vanilla(mfmse, data, max_k=10, name=name)
clear_output(wait=True)
print(results['MAP'])
best_runs

## MLP

In [ ]:
name = 'MLP-BCE'
def init_model(**kwargs):
    return MLP(data.n_user, data.n_item, embedding_dim=128, loss='BCE', **kwargs)

def objective(trial):
    
    # Parameter setup
    params = {'lr': trial.suggest_float('lr', 1e-4, 1e-2, log=True),
              'dropout': trial.suggest_float('dropout', 0, 0.2),
              'n_negatives': trial.suggest_int('neg', 1, 50)}
    params_str = '-'.join(f'{key}={value}' for key, value in params.items())
    data.batch_size = int(base_batch_size / (params['n_negatives'] + 1))
    print(params, data.batch_size)

    # Train
    model = init_model(**params)
    MAP, path, train_likelihood = train(model, data, path='checkpoints/' + name, name=params_str, patience=patience)
    trial.set_user_attr('filename', path)
    trial.set_user_attr('mean_loss', train_likelihood)
    return MAP

study = run_study(name, objective, n_trials=0)
best_runs = study.trials_dataframe().sort_values('value')[::-1]
mlpbce = init_model().load_from_checkpoint(best_runs.user_attrs_filename.iloc[0])
'''
results = test_vanilla(mlpbce, data, max_k=10, name=name)
clear_output(wait=True)
print(results['MAP'])
best_runs
'''

In [ ]:
name = 'MLP-BPR'
def init_model(**kwargs):
    return MLP(data.n_user, data.n_item, embedding_dim=128, loss='BPR', **kwargs)

def objective(trial):
    
    # Parameter setup
    params = {'lr': trial.suggest_float('lr', 1e-4, 1e-2, log=True),
              'dropout': trial.suggest_float('dropout', 0, 0.2)}
    params_str = '-'.join(f'{key}={value}' for key, value in params.items())
    data.batch_size = int(base_batch_size / 2)
    print(params, data.batch_size)

    # Train
    model = init_model(**params)
    MAP, path, train_likelihood = train(model, data, path='checkpoints/' + name, name=params_str, patience=patience)
    trial.set_user_attr('filename', path)
    trial.set_user_attr('mean_loss', train_likelihood)
    return MAP

study = run_study(name, objective, n_trials=trials)
best_runs = study.trials_dataframe().sort_values('value')[::-1]
mlpbpr = init_model().load_from_checkpoint(best_runs.user_attrs_filename.iloc[0])

results = test_vanilla(mlpbpr, data, max_k=10, name=name)
clear_output(wait=True)
print(results['MAP'])
best_runs

In [ ]:
name = 'MLP-MSE'
def init_model(**kwargs):
    return MLP(data.n_user, data.n_item, loss='MSE', **kwargs)

def objective(trial):
    
    # Parameter setup
    params = {'lr': trial.suggest_float('lr', 1e-4, 1e-2, log=True),
              'dropout': trial.suggest_float('dropout', 0, 0.2),
              'n_negatives': trial.suggest_int('neg', 1, 50)}
    params_str = '-'.join(f'{key}={value}' for key, value in params.items())
    data.batch_size = int(base_batch_size / (params['n_negatives'] + 1))
    print(params, data.batch_size)

    # Train
    model = init_model(**params)
    MAP, path, train_likelihood = train(model, data, path='checkpoints/' + name, name=params_str, patience=patience)
    trial.set_user_attr('filename', path)
    trial.set_user_attr('mean_loss', train_likelihood)
    return MAP

study = run_study(name, objective, n_trials=trials)
best_runs = study.trials_dataframe().sort_values('value')[::-1]
model = init_model().load_from_checkpoint(best_runs.user_attrs_filename.iloc[0])

results = test_vanilla(model, data, max_k=10, name=name)
clear_output(wait=True)
print(results['MAP'])
best_runs

## Results

In [ ]:
# Load results
results = {}
for key in os.listdir('results'):
    results[key.replace('.pkl', '').replace('_', ' ')] = pickle.load(open(os.path.join('results', key), 'rb'))
order = ['MF-MSE', 'MF-BCE', 'MF-BPR',
         'MLP-MSE', 'MLP-BCE', 'MLP-BPR']
results = pd.DataFrame([results[key] for key in order], index=order)

# Plot aestetics
colors = [c for c in list(TABLEAU_COLORS)]
colors = {k:c for k, c in zip(results.index, colors)}
lines = ['o', 'v', '^', '<', '>', 's', 'p', '+', 'x', '*', 'p']
lines = {k: '-' + l for k, l, in zip(results.index, lines)}

# Results
print(results['FCP'])

## Top-K accuracy metrics
f, ax = plt.subplots(figsize=(10, 5), ncols=2)
for index, row in results.iterrows():
    ax[0].plot(np.arange(1, 11), row['MAP'], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
    ax[1].plot(np.arange(1, 11), row['Recall'], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
ax[0].set_xticks(np.arange(1, 11))
ax[0].set_xlabel('n', fontsize=20)
ax[0].set_ylabel('MAP@n', fontsize=20)
ax[1].set_xticks(np.arange(1, 11))
ax[1].set_xlabel('n', fontsize=20)
ax[1].set_ylabel('Recall@n', fontsize=20)
handles, labels = ax[0].get_legend_handles_labels()
f.legend(handles, labels, fontsize=15, ncol=3, bbox_to_anchor=(0.83, 1.15))
f.tight_layout()
f.savefig('plots/baselines.pdf', bbox_inches='tight')

# Chapter 4

## User / Item support

In [ ]:
results = test_uncertain(HeuristicUncertainty(baseline=mfbpr, user_uncertainty=-data.user_support), data, name='NUS', max_k=10)
print(results['MAP-Uncertainty'])

In [ ]:
isup = HeuristicUncertainty(baseline=mfbpr, item_uncertainty=-data.item_support)
results = test_uncertain(isup, data, name='NIS', max_k=10)
print(results['MAP-Uncertainty'])

In [ ]:
model = HeuristicUncertainty(baseline=mfbpr, item_uncertainty=-data.item_support)
model.recommend(0)

## Ensemble

In [ ]:
# Load params
best_params = study.best_params
data.batch_size = int(base_batch_size / 2)
best_params = {'weight_decay': best_params['wd']}

# Train
if trials > 0:
    for i in range(4):
        model = init_model(**best_params)
        train(model, data, path='checkpoints/ensemble_MF', name=f'{i}')

# Load ensemble models
models = [mfbpr]
for file in os.listdir('checkpoints/ensemble_MF'):
    models.append(init_model())
    models[-1] = models[-1].load_from_checkpoint(os.path.join('checkpoints/ensemble_MF', file))
    models[-1].eval()
ensemble_mf = Ensemble(models)
clear_output(wait=True)
results = test_uncertain(ensemble_mf, data, name='MF-ENSEMBLE', max_k=10)
print(results['MAP'])

## Network uncertainty

In [ ]:
## MC Dropout
dropout = MCDropout(base_model=mlpbce, mc_iteration=5)
results = test_uncertain(dropout, data, max_k=10, name='MCDropout')
print(results['MAP-Uncertainty'])

In [ ]:
## BayesianNN
name = 'BayesianMLP'
def init_model(**kwargs):
    return BayesianMLP(data.n_user, data.n_item, **kwargs)
base_conf = {'embedding_dim': 10, 'lr': 0, 'n_negatives': 0, 'num_batches': int(len(data.train) / 256)}

def objective(trial):
    
    # Parameter setup
    batch_size = trial.suggest_int('bs', 256, 256)
    params = {'embedding_dim': trial.suggest_int('dim', 128, 128),
              'lr': trial.suggest_float('lr', 1e-4, 1e-2),
              'n_negatives': trial.suggest_int('neg', 20, 20),
              'sample_train': trial.suggest_int('train_samples', 1, 10),
              'prior_pi': trial.suggest_categorical('pi', [1/4, 1/2, 3/4]),
              'prior_sigma_1': trial.suggest_categorical('sigma1', np.exp(-np.array([0, 1, 2]))),
              'prior_sigma_2': trial.suggest_categorical('sigma2', np.exp(-np.array([6, 7, 8])))}
    params['num_batches'] = int(len(data.train) / 256)
    params_str = f'bs = {batch_size}-' + '-'.join(f'{key}={value}' for key, value in params.items())

    # Train
    data.batch_size = batch_size
    model = init_model(**params)
    MAP, path = train(model, data, path='checkpoints/' + name, name=params_str)
    trial.set_user_attr('filename', path)
    return MAP

study = run_study(name, objective, n_trials=5)
BayesMLP, runs = load(init_model(**base_conf), study, top=0)
results = test_uncertain(BayesMLP, data, max_k=10, name=name)

clear_output(wait=True)
print(results['MAP'])
runs[:5]

In [ ]:
## Deep Ensemble
# Load params
best_params = study.best_params
data.batch_size = int(base_batch_size / 2)
best_params = {'dropout': best_params['dropout']}

# Train
if trials > 0:
    for i in range(4):
        model = init_model(**best_params)
        train(model, data, path='checkpoints/ensemble_MLP', name=f'{i}')

# Load ensemble models
models = [mlpbce]
for file in os.listdir('checkpoints/ensemble_MLP'):
    models.append(MLP(data.n_user, data.n_item))
    models[-1] = models[-1].load_from_checkpoint(os.path.join('checkpoints/ensemble_MLP', file))
    models[-1].eval()
ensemble_mlp = Ensemble(models)
clear_output(wait=True)
results = test_uncertain(ensemble_mlp, data, name='MLP-ENSEMBLE', max_k=10)
print(results['MAP-Uncertainty'])

## Results

In [ ]:
# Baseline results
results = {}
for key in os.listdir('results'):
    results[key.replace('.pkl', '').replace('_', ' ')] = pickle.load(open(os.path.join('results', key), 'rb'))
order = ['MF-MSE', 'MF-BCE', 'MF-BPR', 'MLP-MSE', 'MLP-BCE', 'MLP-BPR']
results = pd.DataFrame([results[key] for key in order], index=order)

# Plot aestetics
colors = [c for c in list(TABLEAU_COLORS)]
colors = {k:c for k, c in zip(results.index, colors)}
lines = ['o', 'v', '^', '<', '>', 's', 'p', '+', 'x', '*', 'p']
lines = {k: '-' + l for k, l, in zip(results.index, lines)}

## Baselines
f, ax = plt.subplots(figsize=(10, 5), ncols=2)
for index, row in results.iterrows():
    ax[0].plot(np.arange(1, 11), row['MAP'], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
    ax[1].plot(np.arange(1, 11), row['Recall'], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
ax[0].set_xticks(np.arange(1, 11))
ax[0].set_xlabel('n', fontsize=20)
ax[0].set_ylabel('MAP@n', fontsize=20)
ax[1].set_xticks(np.arange(1, 11))
ax[1].set_xlabel('n', fontsize=20)
ax[1].set_ylabel('Recall@n', fontsize=20)
handles, labels = ax[0].get_legend_handles_labels()
f.legend(handles, labels, fontsize=15, ncol=3, bbox_to_anchor=(0.83, 1.15))
f.tight_layout()
f.savefig('plots/baselines4.pdf', bbox_inches='tight')

# Uncertainty models results
results = {}
for key in os.listdir('results'):
    results[key.replace('.pkl', '').replace('_', ' ')] = pickle.load(open(os.path.join('results', key), 'rb'))
order = ['MF-BPR', 'MLP-BCE', 'NIS', 'MF-ENSEMBLE', 'MLP-ENSEMBLE', 'MCDropout', 'BayesianMLP']
results = pd.DataFrame([results[key] for key in order], index=order)


results.rename(index={'NIS': 'NEG-ITEM-SUPPORT', 'MF-BPR': 'MF', 'MLP-BCE': 'MLP'}, inplace=True)

# Plot aestetics
colors = [c for c in list(TABLEAU_COLORS)]
colors = {k:c for k, c in zip(results.index, colors)}
lines = ['o', 'v', '^', '<', '>', 's', 'p', '+', 'x', '*', 'p']
lines = {k: '-' + l for k, l, in zip(results.index, lines)}

# Correlation plot
f, ax = plt.subplots(figsize=(8, 5), ncols=1)
corr = results[['corr_usup', 'corr_isup']].drop(index=['MF', 'MLP'])
corr.columns=[r'#$R_{u\cdot}$', r'#$R_{\cdot i}$']
corr.loc['NEG-ITEM-SUPPORT', r'#$R_{\cdot i}$'] = np.nan
sns.heatmap(corr.round(3), annot=True, vmax=1, vmin=-1, center=0, cmap='vlag')
plt.yticks(fontsize=15)
plt.xticks(rotation=45, fontsize=20)
plt.tight_layout()
plt.savefig('plots/corr4.pdf')

## Top-K accuracy metrics
f, ax = plt.subplots(figsize=(10, 5), ncols=2)
for index, row in results.iterrows():
    if index != 'NEG-ITEM-SUPPORT':
        ax[0].plot(np.arange(1, 11), row['MAP'], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
        ax[1].plot(np.arange(1, 11), row['Recall'], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
ax[0].set_xticks(np.arange(1, 11))
ax[0].set_xlabel('n', fontsize=20)
ax[0].set_ylabel('MAP@n', fontsize=20)
ax[1].set_xticks(np.arange(1, 11))
ax[1].set_xlabel('n', fontsize=20)
ax[1].set_ylabel('Recall@n', fontsize=20)
handles, labels = ax[0].get_legend_handles_labels()
f.legend(handles, labels, fontsize=15, ncol=3, bbox_to_anchor=(0.86, 1.15))
f.tight_layout()
f.savefig('plots/accuracy4.pdf', bbox_inches='tight')

# MAP vs Uncertainty
results = results.drop(index=['MF', 'MLP'])
f, ax = plt.subplots(figsize=(8, 5))
x = np.arange(10) + 1
for index, row in results.iterrows():
    ax.plot(x, row['MAP-Uncertainty'], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
ax.set(xticks = x)
ax.set_xlabel('Uncertainty bin', fontsize=25)
ax.set_ylabel('MAP@10', fontsize=20)
handles, labels = ax.get_legend_handles_labels()
f.legend(handles, labels, fontsize=15, ncol=3, bbox_to_anchor=(1.038, 1.15))
f.tight_layout()
f.savefig('plots/MAP-Uncertainty.pdf', bbox_inches='tight')

# Cuts
indexs = [index for index in order if index != 'Baseline']
f, ax = plt.subplots(nrows=3, ncols=1, figsize=(5, 10))
for index, row in results.iterrows():
    ax[0].plot(row['Cuts']['MAP'], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
    # if index == 'NIS' or index == 'MF-ENSEMBLE':
    ax[1].plot(1/(1 + np.exp(-row['Cuts']['Relevance'])), lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
    # else:    
    #    ax[1].plot(row['Cuts']['Relevance'], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
    ax[2].plot(row['Cuts']['Coverage'], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
ax[0].set_xticks(range(5))
ax[0].set_xticklabels(np.linspace(1, 0.2, 5).round(2))
ax[0].set_xlabel('Uncertainty quantile cut', fontsize=20)
ax[0].set_ylabel('MAP@10', fontsize=20)
ax[1].set_xticks(range(5))
ax[1].set_xticklabels(np.linspace(1, 0.2, 5).round(2))
ax[1].set_xlabel('Uncertainty quantile cut', fontsize=20)
ax[1].set_ylabel('Mean Predicted Relevance@10', fontsize=20)
ax[2].set_xticks(range(5))
ax[2].set_xticklabels(np.linspace(1, 0.2, 5).round(2))
ax[2].set_xlabel('Uncertainty quantile cut', fontsize=20)
ax[2].set_ylabel('Coverage', fontsize=20)
# handles, labels = ax[0].get_legend_handles_labels()
# f.legend(handles, labels, fontsize=15, bbox_to_anchor=(1.3, 1.1), ncol=5)
f.tight_layout()
f.savefig('plots/cuts4.pdf', bbox_inches="tight")

# UAC / URI
results[['UAC', 'URI']]

In [ ]:
f, ax = plt.subplots(nrows=3, ncols=2, figsize=[15, 10])
idx = [(0, 0), (0, 1), (1, 0), (1, 1), (2, 0)]
model = [isup, ensemble_mf, ensemble_mlp, dropout, dropout]
label = ['NEG-ITEM-SUPPORT', 'MF-ENSEMBLE', 'MLP-ENSEMBLE', 'MCDropout', 'BayesianMLP']
for idx, model, label in zip(idx, model, label):
    if label == 'BayesianMLP':
        rand_preds = model.predict(data.rand['users'][5000:10000], data.rand['items'][5000:10000])
        ax[idx].plot(rand_preds[0], rand_preds[1], 'o')
        ax[idx].set_xlabel('Relevance', fontsize=20)
        ax[idx].set_ylabel('Uncertainty', fontsize=20)
        ax[idx].annotate(label, fontsize=20, xy=(0.55, 0.90), xycoords='axes fraction')
        b, a = np.polyfit(rand_preds[0], rand_preds[1], deg=1)
        xseq = np.linspace(rand_preds[0].min(), rand_preds[0].max(), num=100)
        ax[idx].plot(xseq, a + b * xseq, color="k", lw=2.5)
    else:
        rand_preds = model.predict(data.rand['users'][:5000], data.rand['items'][:5000])
        ax[idx].plot(rand_preds[0], rand_preds[1], 'o')
        ax[idx].set_xlabel('Relevance', fontsize=20)
        ax[idx].set_ylabel('Uncertainty', fontsize=20)
        ax[idx].annotate(label, fontsize=20, xy=(0.55, 0.90), xycoords='axes fraction')
        b, a = np.polyfit(rand_preds[0], rand_preds[1], deg=1)
        xseq = np.linspace(rand_preds[0].min(), rand_preds[0].max(), num=100)
        ax[idx].plot(xseq, a + b * xseq, color="k", lw=2.5)
f.delaxes(ax[2, 1])
f.tight_layout()
f.savefig('plots/relevance_uncertainty.pdf')

# Chapter 5

## DoubleMF

In [ ]:
name = 'AUR_final'
def init_model(**kwargs):
    return JointDoubleMF(data.n_user, data.n_item, embedding_dim=128, embedding_dim_var=128, beta=1e-2, ratio=0.9, loss='AUR', **kwargs)

def objective(trial):
    
    # Parameter setup
    params = {'lr': trial.suggest_float('lr', 1e-5, 1e-3, log=True),
              'beta': trial.suggest_float('beta', 1e-3, 1, log=True),
              'gamma': trial.suggest_float('gamma', 1e-4, 1e-1, log=True),
              'weight_decay': trial.suggest_float('wd', 1e-5, 1e-3),
              'n_negatives': trial.suggest_int('neg', 1, 30)}
    params_str = '-'.join(f'{key}={value}' for key, value in params.items())
    data.batch_size = int(base_batch_size / (params['n_negatives'] + 1))
    print(params, data.batch_size)

    # Train
    model = init_model(**params)
    MAP, path, train_likelihood = train(model, data, path='checkpoints/' + name, name=params_str, patience=patience)
    trial.set_user_attr('filename', path)
    trial.set_user_attr('mean_loss', train_likelihood)

    # Rename model file
    os.rename('checkpoints/' + name + '/last.ckpt', 'checkpoints/' + name + '/' + params_str + '_last.ckpt')
    
    return MAP

study = run_study(name, objective, n_trials=0)
best_runs = study.trials_dataframe().sort_values('value')[::-1][:5]
aur = init_model().load_from_checkpoint(best_runs.user_attrs_filename.iloc[0])
results = test_chap_five(aur, data, max_k=10, name=name)

clear_output(wait=True)
best_runs

## ICPMF

In [ ]:
name = 'CPMF_final'
def init_model(**kwargs):
    return JointDoubleMF(data.n_user, data.n_item, embedding_dim=128, embedding_dim_var=1, beta=1e-2, ratio=0.9, loss='AUR', **kwargs)

def objective(trial):
    
    # Parameter setup
    params = {'lr': trial.suggest_float('lr', 1e-5, 1e-3, log=True),
              'beta': trial.suggest_float('beta', 1e-3, 1, log=True),
              'gamma': trial.suggest_float('gamma', 1e-4, 1e-1, log=True),
              'weight_decay': trial.suggest_float('wd', 1e-5, 1e-3),
              'n_negatives': trial.suggest_int('neg', 1, 30)}
    params_str = '-'.join(f'{key}={value}' for key, value in params.items())
    data.batch_size = int(base_batch_size / (params['n_negatives'] + 1))
    print(params, data.batch_size)

    # Train
    model = init_model(**params)
    MAP, path, train_likelihood = train(model, data, path='checkpoints/' + name, name=params_str, patience=patience)
    print(MAP, path, train_likelihood)
    trial.set_user_attr('filename', path)
    trial.set_user_attr('mean_loss', train_likelihood)

    # Rename model file
    os.rename('checkpoints/' + name + '/last.ckpt', 'checkpoints/' + name + '/' + params_str + '_last.ckpt')
    
    return MAP

study = run_study(name, objective, n_trials=trials)
# best_runs = study.trials_dataframe().sort_values('value')[::-1][:5]
# icmpf = init_model().load_from_checkpoint(best_runs.user_attrs_filename.iloc[0])
icpmf = init_model().load_from_checkpoint('/home/vcoscrato/Documents/RecSys/MF/tests/Pointwise/Movielens/checkpoints/CPMF_final/lr=6.300190162172085e-05-gamma=0.01618271584178488-weight_decay=2.8125637673763354e-05-n_negatives=28-epoch=29-val_MAP=0.1916295737028122.ckpt')
results = test_chap_five(icpmf, data, max_k=10, name=name)

clear_output(wait=True)
# best_runs

## Neural

In [ ]:
name = 'Neural-AUR_final'
def init_model(**kwargs):
    return GaussianMLP(data.n_user, data.n_item, embedding_dim=128, beta=1e-2, ratio=0.9, loss='AUR', **kwargs)

def objective(trial):
    
    # Parameter setup
    params = {'lr': trial.suggest_float('lr', 1e-5, 1e-3, log=True),
              'beta': trial.suggest_float('beta', 1e-3, 1, log=True),
              'gamma': trial.suggest_float('gamma', 1e-4, 1e-1, log=True),
              'dropout': trial.suggest_float('dropout', 0, 0.2),
              'n_negatives': trial.suggest_int('neg', 1, 30)}
    params_str = '-'.join(f'{key}={value}' for key, value in params.items())
    data.batch_size = int(base_batch_size / (params['n_negatives'] + 1))
    print(params, data.batch_size)

    # Train
    model = init_model(**params)
    MAP, path, train_likelihood = train(model, data, path='checkpoints/' + name, name=params_str, patience=patience)
    trial.set_user_attr('filename', path)
    trial.set_user_attr('mean_loss', train_likelihood)
    return MAP

study = run_study(name, objective, n_trials=trials)
best_runs = study.trials_dataframe().sort_values('value')[::-1][:5]
gaussianMLP = init_model().load_from_checkpoint(best_runs.user_attrs_filename.iloc[0])
gaussianMLP.dropl.eval()
results = test_chap_five(gaussianMLP, data, max_k=10, name=name)

clear_output(wait=True)
best_runs

## Hierarchical Bayesian Ranking (HBR)

In [ ]:
from uncertain.implicit.pyro import HBR, train, visualize

hbr = HBR(data.n_user, data.n_item, embedding_dim=128)
visualize(hbr, data)

In [ ]:
name = 'HBR_final'
def init_model(**kwargs):
    return HBR(data.n_user, data.n_item, embedding_dim=128, **kwargs)

def objective(trial):
    pyro.clear_param_store()
    
    # Parameter setup
    tau_ui = trial.suggest_float('tau_gamma', 1e-5, 1e-1, log=True)
    params = {'lr': trial.suggest_float('lr', 1e-5, 1e-3, log=True),
              'tau_gamma': trial.suggest_float('tau_gamma', 1e-5, 1e-1, log=True),
              'tau_u': tau_ui,
              'tau_i': tau_ui,
              'n_negatives': trial.suggest_int('neg', 1, 30)}
    params_str = '-'.join(f'{key}={value}' for key, value in params.items())
    data.batch_size = int(base_batch_size / (params['n_negatives'] + 1))
    print(params, data.batch_size)

    # Train
    model = init_model(**params)
    MAP = train(model, data, n_steps=200, val_every_n_epochs=2)
    filename = 'checkpoints/Bern_CPMF/' + params_str + '.pkl'

    # Save model params
    with open(filename, 'wb') as f:
        params = {'user_embeddings': model.user_embeddings,
                  'item_embeddings': model.item_embeddings,
                  'user_var': model.user_var,
                  'item_var': model.item_var}
        pickle.dump(params, f, protocol=5)
    
    trial.set_user_attr('filename', filename)
    return MAP

study = run_study(name, objective, n_trials=0)
best_runs = study.trials_dataframe().sort_values('value')[::-1]
best_model_path = study.trials_dataframe().sort_values('value')['user_attrs_filename'].iloc[-1]
with open(best_model_path, 'rb') as f:
    params = pickle.load(f)
    bern_cpmf = init_model()
    bern_cpmf.user_embeddings = params['user_embeddings']
    bern_cpmf.item_embeddings = params['item_embeddings']
    bern_cpmf.user_var = params['user_var']
    bern_cpmf.item_var = params['item_var']

results = test_chap_five(bern_cpmf, data, max_k=10, name='HBR-R', debug=True)
clear_output(wait=True)
print(results['MAP'].mean(0)[0], results['MAP'].mean(0)[-1])

import copy
hbrY = copy.copy(bern_cpmf)
hbrY.interact = hbrY.interact_return_logit_normal

results = test_chap_five(hbrY, data, max_k=10, name='HBR-Y', debug=True)
clear_output(wait=True)
print(results['MAP'].mean(0)[0], results['MAP'].mean(0)[-1])

best_runs

## Results

In [ ]:
results = {}
for key in os.listdir('results'):
    results[key.replace('.pkl', '').replace('_final', '')] = pickle.load(open(os.path.join('results', key), 'rb'))
order = ['MF-BPR', 'AUR', 'CPMF', 'Neural-AUR', 'HBR', 'HBR-Y']
results = pd.DataFrame([results[key] for key in order], index=order)
results.rename(index={'Neural-AUR': 'Gaussian-MLP', 'CPMF': 'ICPMF', 'HBR': 'HBR-R', 'MF-BPR': 'MF (Baseline)'}, inplace=True)

# Plot aestetics
colors = [c for c in list(TABLEAU_COLORS)]
colors = {k:c for k, c in zip(results.index, colors)}
lines = ['o', 'v', '^', '<', '>', 's', 'p', '+', 'x', '*', 'p']
lines = {k: '-' + l for k, l, in zip(results.index, lines)}

## Baselines
f, ax = plt.subplots(figsize=(10, 4), ncols=2)
for index, row in results.iterrows():
    if index == 'MF (Baseline)':
        ax[0].plot(np.repeat(row['MAP'][-1], 11), '--', color=colors[index], label=index, linewidth=3, alpha=0.6)
        ax[1].plot(np.repeat(row['Recall'][-1], 11), '--', color=colors[index], label=index, linewidth=3, alpha=0.6)
    else:
        ax[0].plot(row['MAP'].mean(0)[:, -1], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
        ax[1].plot(row['Recall'].mean(0)[:, -1], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
for line in [0, 1]:
    ax[line].set_xticks(np.arange(11))
    ax[line].set_xticklabels(np.linspace(0, 1, 11).round(2))
    ax[line].set_xlabel(r'$\lambda$', fontsize=20)
ax[0].set_ylabel('MAP@10', fontsize=20)
ax[1].set_ylabel('Recall@10', fontsize=20)
handles, labels = ax[0].get_legend_handles_labels()
f.legend(handles, labels, fontsize=15, ncol=3, bbox_to_anchor=(0.851, 1.2))
f.tight_layout()
f.savefig('plots/accuracy5.pdf', bbox_inches='tight')


f, ax = plt.subplots(figsize=(10, 4), ncols=2)
for index, row in results.iterrows():
    if index == 'MF (Baseline)':
        ax[0].plot(np.repeat(row['Map relative'][1] -0.05, 11), '--', color=colors[index], label=index, linewidth=3, alpha=0.6)
        ax[1].plot(np.repeat(row['Recall relative'][3] -0.014, 11), '--', color=colors[index], label=index, linewidth=3, alpha=0.6)
    else:
        ax[0].plot(np.nanmean(row['Map relative'], 0)[:, -1], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
        ax[1].plot(np.nanmean(row['Recall relative'], 0)[:, -1], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
for line in [0, 1]:
    ax[line].set_xticks(np.arange(0, 11))
    ax[line].set_xticklabels(np.linspace(0, 1, 11).round(2))
    ax[line].set_xlabel(r'$\lambda$', fontsize=20)
ax[0].set_ylabel('MAP Relative@10', fontsize=20)
ax[1].set_ylabel('Recall Relative@10', fontsize=20)
handles, labels = ax[0].get_legend_handles_labels()
f.legend(handles, labels, fontsize=15, ncol=3, bbox_to_anchor=(0.851, 1.2))
f.tight_layout()
f.savefig('plots/accuracy_relative5.pdf', bbox_inches='tight')


# # Correlation plot
# f, ax = plt.subplots(figsize=(8, 5), ncols=1)
# corr = results[['corr_usup', 'corr_isup']]
# corr.columns=[r'#$R_{u\cdot}$', r'#$R_{\cdot i}$']
# # corr.loc['NEG-ITEM-SUPPORT', r'#$R_{\cdot i}$'] = np.nan
# sns.heatmap(corr.round(3), annot=True, vmax=1, vmin=-1, center=0, cmap='vlag')
# plt.yticks(fontsize=15)
# plt.xticks(rotation=45, fontsize=20)
# plt.tight_layout()
# plt.savefig('plots/corr5.pdf')

# # Ratio
# indexs = [index for index in order if index != 'Baseline']
# f, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
# for index, row in results.iterrows():
#     ax[0].plot(row['MAP'].mean(0)[:, -1], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
#     ax[1].plot(row['Recall'].mean(0)[:, -1], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
# ax[0].set_xticks(np.arange(0, 11))
# ax[0].set_xticklabels(np.linspace(0, 1, 11).round(2))
# ax[0].set_xlabel(r'$\lambda$', fontsize=20)
# ax[0].set_ylabel('MAP@10', fontsize=20)
# ax[1].set_xticks(np.arange(0, 11))
# ax[1].set_xticklabels(np.linspace(0, 1, 11).round(2))
# ax[1].set_xlabel(r'$\lambda$', fontsize=20)
# ax[1].set_ylabel('Recall@10', fontsize=20)
# f.tight_layout()
# f.savefig('plots/accuracy_ratio.pdf', bbox_inches="tight")

In [ ]:
f, ax = plt.subplots(nrows=3, ncols=2, figsize=[12, 10])
idx = [(0, 0), (0, 1), (1, 0), (1, 1), (2, 0)]
model = [aur, icpmf, gaussianMLP, bern_cpmf, 0]
label = ['AUR', 'ICPMF', 'Gaussian-MLP', 'HBR-R', 'HBR-Y']
for idx, model, label in zip(idx, model, label):
    if label == 'HBR-Y':
        mu, var = rand_preds[0], rand_preds[1]
        constant =  3 / torch.pi**2
        denom = np.sqrt(1 + constant * var)
        mu_logistic_normal = 1 / (1 + np.exp(-mu / denom))
        var_logistic_normal = mu_logistic_normal * (1 - mu_logistic_normal) * (1 - 1 / denom)
        rand_preds = [mu_logistic_normal, var_logistic_normal]
        ax[idx].set_xlabel(r'Relevance: $E[P_{ui}]$', fontsize=20)
        ax[idx].set_ylabel(r'Uncertainty: $Var[P_{ui}]$', fontsize=20)
    else:
        rand_preds = model.predict(data.rand['users'][:5000], data.rand['items'][:5000])
        ax[idx].set_xlabel(r'Relevance: $\mu_{ui}$', fontsize=20)
        ax[idx].set_ylabel(r'Uncertainty: $\sigma^2_{ui}$', fontsize=20)
    if not 'HBR' in label:
        rand_preds = rand_preds[0], np.exp(rand_preds[1])
    idx_valid = np.logical_and(rand_preds[1] > -5, rand_preds[1] < 5)
    ax[idx].plot(rand_preds[0][idx_valid], rand_preds[1][idx_valid], 'o')
    ax[idx].annotate(label, fontsize=20, xy=(0.55, 0.90), xycoords='axes fraction')
    b, a = np.polyfit(rand_preds[0][idx_valid], rand_preds[1][idx_valid], deg=1)
    xseq = np.linspace(rand_preds[0][idx_valid].min(), rand_preds[0][idx_valid].max(), num=100)
    ax[idx].plot(xseq, a + b * xseq, color="k", lw=2.5)


f.delaxes(ax[2, 1])
f.tight_layout()
f.savefig('plots/relevance_uncertainty5.pdf')

# Not used

This is some extra stuff tested during development that were not used in our papers.

## MAP based CPMF (not fully bayesian)

In [ ]:
class CPMF(Implicit, UncertainRecommender):

    def __init__(self, n_user, n_item, embedding_dim):

        super().__init__()
        self.n_user = n_user
        self.n_item = n_item
        self.embedding_dim = embedding_dim
        self.softplus = torch.nn.Softplus()

    def model(self, user, item, Y=None):
        # Obs noise precision (alpha in the papers)
        obs_noise = torch.tensor(2.)
        # Embeddings precision (alpha_u, alpha_i) in the paper
        # Higher values -> More precision -> Less variance -> More reguralization
        τ_u = torch.tensor(.004)
        τ_i = torch.tensor(.004)

        # CPMF precision params
        γ_u = pyro.param('γ_u', torch.ones(self.n_user))
        γ_i = pyro.param('γ_i', torch.ones(self.n_item))
        
        p_u = pyro.sample('p_u', dist.Normal(torch.zeros(self.n_user, self.embedding_dim), 1/τ_u.sqrt()).to_event(2))
        q_i = pyro.sample('q_i', dist.Normal(torch.zeros(self.n_item, self.embedding_dim), 1/τ_u.sqrt()).to_event(2))

        with pyro.plate('Relevance', len(user)):
            mean = (p_u[user] * q_i[item]).sum(1)
            precision = self.softplus(γ_u[user] * γ_i[item] * obs_noise)
            r_ui = pyro.sample('r_ui', dist.Normal(mean, 1/precision.sqrt()), obs=torch.tensor(Y).float())

    def guide(self, user, item, Y=None):
        # Init the embeddings according to N(0, 1/d)
        µ_u = pyro.param('µ_u', torch.randn(self.n_user, self.embedding_dim) / self.embedding_dim)
        µ_i = pyro.param('µ_i', torch.randn(self.n_item, self.embedding_dim) / self.embedding_dim)

        p_u = pyro.sample('p_u', dist.Delta(µ_u).to_event(2))
        q_i = pyro.sample('q_i', dist.Delta(µ_i).to_event(2))

    def fix_learned_embeddings(self):
        model.user_embeddings = pyro.param('µ_u').detach()
        model.item_embeddings = pyro.param('µ_i').detach()
        self.user_var = pyro.param('γ_u').detach()
        self.item_var = pyro.param('γ_i').detach()

    def get_user_embeddings(self, user_ids):
        return self.user_embeddings[user_ids], self.user_var[user_ids]
    
    def get_item_embeddings(self, item_ids=None):
        if item_ids is not None:
            return self.item_embeddings[item_ids], self.item_var[item_ids]
        else:
            return self.item_embeddings, self.item_var
    
    def interact(self, user_embeddings, item_embeddings):
        relevance = (user_embeddings[0] * item_embeddings[0]).sum(1)
        unc = 1 / self.softplus(user_embeddings[1] * item_embeddings[1])
        return relevance, unc
                
                

pyro.clear_param_store()
model = CPMF(data.n_user, data.n_item, 16)
visualize(model)

In [ ]:
pyro.clear_param_store()
train(model, data, n_negative=50, n_steps=10, lr=0.0005)
clear_output(wait=True)
plt.plot(np.log(model.epoch_loss))

for i in pyro.get_param_store():
    print(i, pyro.param(i))

rec = model.recommend(3, n=data.n_item)
rec['support'] = data.item['support'].loc[rec.index]
rec

In [ ]:
results = test_chap_five(model, data, max_k=10, name='SVI_CPMF', debug=False)
clear_output(wait=True)
results['MAP'].mean(0)[0], results['MAP'].mean(0)[-1]

## Add the Bernoulli layer

In [ ]:
results['Map relative'].mean(0)[0], results['Map relative'].mean(0)[-1]

In [ ]:
from pyro.infer import Predictive

model.predictive = True
predictive = Predictive(model.model, guide=model.guide, num_samples=10, return_sites=('p_u', 'q_i'))
samples = predictive(data.train[:, 0][:2], data.train[:, 1][:2])

In [ ]:
mu_grid = [-10, 0, 10]
f, ax = plt.subplots(figsize=(10, 5), ncols=len(mu_grid))
for idx, mu in enumerate(mu_grid):
    sample = torch.randn(10000).numpy() + mu 
    transformed = 1 / (1 + np.exp(-sample))
    ax[idx].hist(transformed)

In [ ]:
mu = 1
var_grid = [0.1, 1, 2, 3, 5]
f, ax = plt.subplots(figsize=(15, 5), ncols=len(var_grid))
for idx, var in enumerate(var_grid):
    sample = (torch.randn(100000).numpy() + mu) * np.sqrt(var) 
    transformed = 1 / (1 + np.exp(-sample))
    ax[idx].hist(transformed, bins=50, density=True)
    ax[idx].set_xlabel(r'$P(Y_{ui} = 1)$')
    ax[idx].set_title(r'$r_{ui}$ Variance: ' + str(var))
f.tight_layout()

In [ ]:
var_grid = [0.1, 1, 2, 5]
f, ax = plt.subplots(figsize=(15, 5), ncols=len(var_grid))
for idx, var in enumerate(var_grid):
    sample = (torch.randn(10000).numpy() - 2) * np.sqrt(var) 
    ax[idx].hist(sample)
    ax[idx].set_title('r_ui Variance: {}'.format(var))

## CPMF with introduced priors for auto-regularization

In [ ]:
class Bern_CPMF(Implicit, UncertainRecommender):

    def __init__(self, n_user, n_item, embedding_dim, lr=0.0001, n_negatives=10, tau_f=1e-2, tau_g=1e-2):

        super().__init__()
        self.n_user = n_user
        self.n_item = n_item
        self.embedding_dim = embedding_dim
        self.lr = lr
        self.n_negatives = n_negatives
        self.softplus = torch.nn.Softplus()
        self.sigmoid = lambda x: 1 / (1 + torch.exp(-x))
        self.sig_trans = dist.transforms.SigmoidTransform()
        
        self.predictive = False

        # Embeddings precision (alpha_u, alpha_i) in the paper
        # Higher values -> More precision -> Less variance -> More reguralization
        self.τ_f = torch.tensor(tau_f)
        self.τ_g = torch.tensor(tau_g)
    
    def model(self, user, item, Y=None):
        if Y is not None:
            Y = torch.tensor(Y).float()
        
        p_u = pyro.sample('p_u', dist.Normal(torch.zeros(self.n_user, self.embedding_dim), 1/self.τ_f.sqrt()).to_event(2))
        q_i = pyro.sample('q_i', dist.Normal(torch.zeros(self.n_item, self.embedding_dim), 1/self.τ_f.sqrt()).to_event(2))

        γ_u = pyro.sample('γ_u', dist.Normal(torch.zeros(self.n_user), 1/self.τ_g.sqrt()).to_event(1))
        γ_i = pyro.sample('γ_i', dist.Normal(torch.zeros(self.n_item), 1/self.τ_g.sqrt()).to_event(1))

        with pyro.plate('Relevance', len(user)):
            mean = (p_u[user] * q_i[item]).sum(1)
            γ_ui = γ_u[user] * γ_i[item]
            sqrt_var_ui = torch.sqrt(1 / self.softplus(γ_ui))
            prob_ui = pyro.sample('prob_ui', dist.TransformedDistribution(dist.Normal(mean, sqrt_var_ui), [self.sig_trans]))
            Y_ui = pyro.sample('Y_ui', dist.Bernoulli(prob_ui), obs=Y)

    def guide(self, user, item, Y=None):
        
        # Init the embeddings according to N(0, 1/d)
        µ_u = pyro.param('µ_u', torch.randn(self.n_user, self.embedding_dim) / self.embedding_dim)
        µ_i = pyro.param('µ_i', torch.randn(self.n_item, self.embedding_dim) / self.embedding_dim)
        
        γ_u_param = pyro.param('γ_u_param', torch.randn(self.n_user))
        γ_i_param = pyro.param('γ_i_param', torch.randn(self.n_item))

        p_u = pyro.sample('p_u', dist.Delta(µ_u).to_event(2))
        q_i = pyro.sample('q_i', dist.Delta(µ_i).to_event(2))
        γ_u = pyro.sample('γ_u', dist.Delta(γ_u_param).to_event(1))
        γ_i = pyro.sample('γ_i', dist.Delta(γ_i_param).to_event(1))

        if self.predictive:
            return None
        
        with pyro.plate('Relevance', len(user)):
            mean = (p_u[user] * q_i[item]).sum(1)
            γ_ui = γ_u[user] * γ_i[item]
            sqrt_var_ui = torch.sqrt(1 / self.softplus(γ_ui))
            prob_ui = pyro.sample('prob_ui', dist.TransformedDistribution(dist.Normal(mean, sqrt_var_ui), [self.sig_trans]))

    def fix_learned_embeddings(self):
        self.user_embeddings = pyro.param('µ_u').detach()
        self.item_embeddings = pyro.param('µ_i').detach()
        self.user_var = pyro.param('γ_u_param').detach()
        self.item_var = pyro.param('γ_i_param').detach()

    def get_user_embeddings(self, user_ids):
        return self.user_embeddings[user_ids], self.user_var[user_ids]
    
    def get_item_embeddings(self, item_ids=None):
        if item_ids is not None:
            return self.item_embeddings[item_ids], self.item_var[item_ids]
        else:
            return self.item_embeddings, self.item_var

    def interact(self, user_embeddings, item_embeddings):
        # This interaction function returns the mean and variance for the gaussian hidden variable
        mu = (user_embeddings[0] * item_embeddings[0]).sum(1)
        var = 1 / torch.exp(user_embeddings[1] * item_embeddings[1])
        return mu, var

    def uncertain_transform(self, obj):
        mu, var = obj
        # Logistic normal approximated analytical form for the mean
        constant =  3 / torch.pi**2
        denom = torch.sqrt(1 + constant * var)
        mu_logistic_normal = self.sigmoid(mu / denom)
        return mu_logistic_normal
    
    def interact_return_logit_normal(self, user_embeddings, item_embeddings):
        mu, var = interact(user_embeddings, item_embeddings)
        
        # Logistic normal approximated analytical forms
        constant =  3 / torch.pi**2
        denom = torch.sqrt(1 + constant * var)
        mu_logistic_normal = self.sigmoid(mu / denom)
        var_logistic_normal = mu_logistic_normal * (1 - mu_logistic_normal) * (1 - 1 / denom)
        return mu_logistic_normal, var_logistic_normal


pyro.clear_param_store()
model = Bern_CPMF(data.n_user, data.n_item, 16)
visualize(model, False)

In [ ]:
name = 'Bern_CPMF2'
def init_model(**kwargs):
    return Bern_CPMF(data.n_user, data.n_item, embedding_dim=128, **kwargs)

def objective(trial):
    pyro.clear_param_store()
    
    # Parameter setup
    params = {'lr': trial.suggest_float('lr', 5e-5, 5e-5, log=True),
              'tau_f': trial.suggest_float('tau_f', 1e-4, 1e-4, log=True),
              'tau_g': trial.suggest_float('tau_g', 1, 1, log=True),
              'n_negatives': trial.suggest_int('neg', 1, 1)}
    params_str = '-'.join(f'{key}={value}' for key, value in params.items())
    data.batch_size = int(base_batch_size / (params['n_negatives'] + 1))
    print(params, data.batch_size)

    # Train
    model = init_model(**params)
    MAP = train(model, data, n_steps=200, val_every_n_epochs=2)
    filename = 'checkpoints/Bern_CPMF2/' + params_str + '.pkl'

    # Save model params
    with open(filename, 'wb') as f:
        params = {'user_embeddings': model.user_embeddings,
                  'item_embeddings': model.item_embeddings,
                  'user_var': model.user_var,
                  'item_var': model.item_var}
        pickle.dump(params, f, protocol=5)
    
    trial.set_user_attr('filename', filename)
    return MAP

study = run_study(name, objective, n_trials=1)
best_runs = study.trials_dataframe().sort_values('value')[::-1]
best_model_path = study.trials_dataframe().sort_values('value')['user_attrs_filename'].iloc[-1]
with open(best_model_path, 'rb') as f:
    params = pickle.load(f)
    bern_cpmf = init_model()
    bern_cpmf.user_embeddings = params['user_embeddings']
    bern_cpmf.item_embeddings = params['item_embeddings']
    bern_cpmf.user_var = params['user_var']
    bern_cpmf.item_var = params['item_var']

results = test_chap_five(bern_cpmf, data, max_k=10, name=name, debug=True)
clear_output(wait=True)
print(results['MAP'].mean(0)[0], results['MAP'].mean(0)[-1])
best_runs

In [ ]:

class CPMF(Implicit, UncertainRecommender):

    def __init__(self, n_user, n_item, embedding_dim):

        super().__init__()
        self.n_user = n_user
        self.n_item = n_item
        self.embedding_dim = embedding_dim
        self.softplus = torch.nn.Softplus()

    def model(self, user, item, Y=None):
        # Obs noise precision (alpha in the papers)
        obs_noise = torch.tensor(2.)
        # Embeddings precision (alpha_u, alpha_i) in the paper
        # Higher values -> More precision -> Less variance -> More reguralization
        τ_u = torch.tensor(.004)
        τ_i = torch.tensor(.004)

        # CPMF precision params
        γ_u = pyro.param('γ_u', torch.ones(self.n_user))
        γ_i = pyro.param('γ_i', torch.ones(self.n_item))
        
        p_u = pyro.sample('p_u', dist.Normal(torch.zeros(self.n_user, self.embedding_dim), 1/τ_u.sqrt()).to_event(2))
        q_i = pyro.sample('q_i', dist.Normal(torch.zeros(self.n_item, self.embedding_dim), 1/τ_u.sqrt()).to_event(2))

        with pyro.plate('Relevance', len(user)):
            mean = (p_u[user] * q_i[item]).sum(1)
            precision = self.softplus(γ_u[user] * γ_i[item] * obs_noise)
            r_ui = pyro.sample('r_ui', dist.Normal(mean, 1/precision.sqrt()), obs=torch.tensor(Y).float())

    def guide(self, user, item, Y=None):
        # Init the embeddings according to N(0, 1/d)
        µ_u = pyro.param('µ_u', torch.randn(self.n_user, self.embedding_dim) / self.embedding_dim)
        µ_i = pyro.param('µ_i', torch.randn(self.n_item, self.embedding_dim) / self.embedding_dim)

        p_u = pyro.sample('p_u', dist.Delta(µ_u).to_event(2))
        q_i = pyro.sample('q_i', dist.Delta(µ_i).to_event(2))

    def fix_learned_embeddings(self):
        model.user_embeddings = pyro.param('µ_u').detach()
        model.item_embeddings = pyro.param('µ_i').detach()
        self.user_var = pyro.param('γ_u').detach()
        self.item_var = pyro.param('γ_i').detach()

    def get_user_embeddings(self, user_ids):
        return self.user_embeddings[user_ids], self.user_var[user_ids]
    
    def get_item_embeddings(self, item_ids=None):
        if item_ids is not None:
            return self.item_embeddings[item_ids], self.item_var[item_ids]
        else:
            return self.item_embeddings, self.item_var
    
    def interact(self, user_embeddings, item_embeddings):
        relevance = (user_embeddings[0] * item_embeddings[0]).sum(1)
        unc = 1 / self.softplus(user_embeddings[1] * item_embeddings[1])
        return relevance, unc
                
                

pyro.clear_param_store()
model = CPMF(data.n_user, data.n_item, 16)
visualize(model)







    model.user_embeddings = pyro.param('µ_u').detach()
    model.item_embeddings = pyro.param('µ_i').detach()
    model.user_var = pyro.param('α_γu').detach() / pyro.param('β_γu').detach()
    model.item_var = pyro.param('α_γi').detach() / pyro.param('β_γi').detach()



class CPMF_priors(Implicit, UncertainRecommender):

    def __init__(self, n_user, n_item, embedding_dim, n_negative=20, init_lr=0.0001):

        super().__init__()
        self.n_user = n_user
        self.n_item = n_item
        self.embedding_dim = embedding_dim
        self.learning_rate = init_lr
        self.n_negative = n_negative
        
        self.alpha = torch.tensor(2.)
        self.alpha_u = torch.tensor(0.001)
        self.alpha_i = torch.tensor(0.001)

        self.logit = lambda x: 1 / (1 + torch.exp(-x))

    def model(self, user, item, Y=None):

        # Hyperparams
        ## Obs noise (α in the papers)
        ε = self.alpha
        ## Embeddings averages
        µ_u0 = torch.zeros(self.embedding_dim)
        µ_i0 = torch.zeros(self.embedding_dim)
        ## Embedding precisions (regularizer)
        ## α > β -> higher precision mean = less precision variance -> more stable embeddings
        ## α / β grows -> less gamma variance -> more stable embeddings
        α_pu0 = torch.tensor(10.)
        β_pu0 = torch.tensor(10.)
        α_qi0 = torch.tensor(10.)
        β_qi0 = torch.tensor(10.)
        ## CPMF variance gamma hyperparams
        α_γu0 = torch.tensor(10.)
        β_γu0 = torch.tensor(10.)
        α_γi0 = torch.tensor(10.)
        β_γi0 = torch.tensor(10.)

        # Hyperpriors (α_u and α_v in the papers)
        τ_pu = pyro.sample('τ_pu', dist.Gamma(concentration=α_pu0, rate=β_pu0))
        τ_qi = pyro.sample('τ_qi', dist.Gamma(concentration=α_qi0, rate=β_qi0))

        # Priors
        γ_u = pyro.sample('γ_u', dist.Gamma(α_γu0, β_γu0).expand([self.n_user]).to_event(1))
        γ_i = pyro.sample('γ_i', dist.Gamma(α_γi0, β_γi0).expand([self.n_item]).to_event(1))
        with pyro.plate('Users', self.n_user):
            p_u = pyro.sample('p_u', dist.Normal(µ_u0, 1/τ_pu.sqrt()).to_event(1))
        with pyro.plate('Items', self.n_item):
            q_i = pyro.sample('q_i', dist.Normal(µ_i0, 1/τ_qi.sqrt()).to_event(1))

        with pyro.plate('Relevance', len(user)):
            mean = (p_u[user] * q_i[item]).sum(1)
            precision = γ_u[user] * γ_i[item] * ε
            r_ui = pyro.sample('r_ui', dist.Normal(mean, 1/(precision.sqrt())))
            Y_ui = pyro.sample('Y_ui', dist.Bernoulli(self.logit(r_ui)), obs=torch.tensor(Y).float())

        # Debug
        # f, ax = plt.subplots(ncols=3)
        # var = 1 / (precision)
        # print('Precision:', precision.detach().numpy().min(), precision.detach().numpy().max(), precision.detach().numpy().mean())
        # print('Variance:', var.detach().numpy().min(), var.detach().numpy().max(), var.detach().numpy().mean())
        # ax[0].hist(precision.sqrt().detach().numpy(), bins=100, color='green', label='precision sqrt')
        # ax[1].hist(var.detach().numpy(), bins=100, color='green', label='variance')
        # ax[2].hist(r_ui.detach().numpy(), bins=100, color='green', label='relevance')

    def guide(self, user, item, Y=None):

        # Variational params
        ## Obs noise (α in the papers)
        # ε = pyro.param('ε', torch.tensor(2), constraint=constraints.positive)
        ## Embeddings
        µ_u = pyro.param('µ_u', torch.randn(self.n_user, self.embedding_dim) /1000)
        µ_i = pyro.param('µ_i', torch.randn(self.n_item, self.embedding_dim) /1000)
        ## Embedding vars (regularizer)
        α_pu = pyro.param('α_pu', torch.tensor(50000.), constraint=constraints.positive)
        β_pu = pyro.param('β_pu', torch.tensor(50000.), constraint=constraints.positive)
        α_qi = pyro.param('α_qi', torch.tensor(50000.), constraint=constraints.positive)
        β_qi = pyro.param('β_qi', torch.tensor(50000.), constraint=constraints.positive)
        ## CPMF gamma hyperparams - Related to precision
        α_γu = pyro.param('α_γu', torch.zeros(self.n_user)+5000., constraint=constraints.positive)
        β_γu = pyro.param('β_γu', torch.zeros(self.n_user)+1000., constraint=constraints.positive)
        α_γi = pyro.param('α_γi', torch.zeros(self.n_item)+5000., constraint=constraints.positive)
        β_γi = pyro.param('β_γi', torch.zeros(self.n_item)+1000., constraint=constraints.positive)
        
        # Hyperpriors (α_u and α_v in the papers)
        τ_pu = pyro.sample('τ_pu', dist.Gamma(α_pu, β_pu))
        τ_qi = pyro.sample('τ_qi', dist.Gamma(α_qi, β_qi))

        # Priors
        γ_u = pyro.sample('γ_u', dist.Gamma(α_γu, β_γu).to_event(1))
        γ_i = pyro.sample('γ_i', dist.Gamma(α_γi, β_γi).to_event(1))
        with pyro.plate('Users'):
            p_u = pyro.sample('p_u', dist.Normal(µ_u, 1/τ_pu.sqrt()).to_event(1))
        with pyro.plate('Items'):
            q_i = pyro.sample('q_i', dist.Normal(µ_i, 1/τ_qi.sqrt()).to_event(1))

        with pyro.plate('Relevance', len(user)):
            mean = (p_u[user] * q_i[item]).sum(1)
            precision = γ_u[user] * γ_i[item]
            r_ui = pyro.sample('r_ui', dist.Normal(mean, 1/precision.sqrt()))

        # # Debug
        # f, ax = plt.subplots(ncols=3)
        # var = 1 / (precision)
        # print('Precision:', precision.detach().numpy().min(), precision.detach().numpy().max(), precision.detach().numpy().mean())
        # print('Variance:', var.detach().numpy().min(), var.detach().numpy().max(), var.detach().numpy().mean())
        # ax[0].hist(precision.sqrt().detach().numpy(), bins=100, color='green', label='precision sqrt')
        # ax[1].hist(var.detach().numpy(), bins=100, color='green', label='variance')
        # ax[2].hist(r_ui.detach().numpy(), bins=100, color='green', label='relevance')

    def guide2(self, user, item, Y=None):

        # Variational params
        ## Gammas
        µ_γ_u = pyro.param('µ_γ_u', torch.ones(self.n_user))
        µ_γ_i = pyro.param('µ_γ_i', torch.ones(self.n_item))
        var_gamma_u = pyro.param('var_γ_u', torch.zeros(self.n_user) +.01, constraint=constraints.positive)
        var_gamma_i = pyro.param('var_γ_i', torch.zeros(self.n_item) +.01, constraint=constraints.positive)
        ## Embeddings
        µ_u = pyro.param('µ_u', torch.zeros(self.n_user, self.embedding_dim) +.1)
        µ_i = pyro.param('µ_i', torch.zeros(self.n_item, self.embedding_dim) +.1)
        ## Embedding vars (regularizer)
        mu_alpha_u = pyro.param('µ_alpha_u', torch.tensor(1.))
        mu_alpha_i = pyro.param('µ_alpha_i', torch.tensor(1.))
        var_alpha_u = pyro.param('var_alpha_u', torch.tensor(.01), constraint=constraints.positive)
        var_alpha_i = pyro.param('var_alpha_i', torch.tensor(.01), constraint=constraints.positive)
        
        # Hyperpriors
        ## Embedding vars (regularizer)
        α_pu = pyro.sample('α_pu', dist.LogNormal(mu_alpha_u, var_alpha_u))
        α_qi = pyro.sample('α_qi', dist.LogNormal(mu_alpha_i, var_alpha_i))
        γ_u = pyro.sample('γ_u', dist.LogNormal(µ_γ_u, var_gamma_u).expand([self.n_user]).to_event(1))
        γ_i = pyro.sample('γ_i', dist.LogNormal(µ_γ_i, var_gamma_i).expand([self.n_item]).to_event(1))

        # Priors
        with pyro.plate('Users'):
            p_u = pyro.sample('p_u', dist.Normal(µ_u, α_pu).to_event(1))
        with pyro.plate('Items'):
            q_i = pyro.sample('q_i', dist.Normal(µ_i, α_qi).to_event(1))

        with pyro.plate('Relevance', len(user)):
            mean = (p_u[user] * q_i[item]).sum(1)
            var = torch.exp(γ_u[user] * γ_i[item])
            r_ui = pyro.sample('r_ui', dist.Normal(mean, var))

    def get_user_embeddings(self, user_ids):
        return self.user_embeddings[user_ids], self.user_var[user_ids]
    
    def get_item_embeddings(self, item_ids=None):
        if item_ids is not None:
            return self.item_embeddings[item_ids], self.item_var[item_ids]
        else:
            return self.item_embeddings, self.item_var
    
    def interact(self, user_embeddings, item_embeddings):
        relevance = self.logit((user_embeddings[0] * item_embeddings[0]).sum(1))
        unc = 1 / (user_embeddings[1] * item_embeddings[1])
        return relevance, unc
                

pyro.clear_param_store()
model = CPMF_like(data.n_user, data.n_item, embedding_dim=2)

display(pyro.render_model(model.model, model_args=(data.train[:, 0], 
                                                   data.train[:, 1], 
                                                   np.zeros_like(data.train[:, 0])), 
                          render_params=True, render_distributions=True))

# pyro.infer.autoguide.AutoNormal(model.model)
display(pyro.render_model(model.guide, model_args=(data.train[:, 0], 
                                                   data.train[:, 1], 
                                                   np.ones_like(data.train[:, 0])), 
                          render_params=True, render_distributions=True))

# trace = poutine.trace(model.guide).get_trace(data.train[:, 0], 
#                                              data.train[:, 1], 
#                                              np.ones_like(data.train[:, 0]))
print(trace.format_shapes())

In [ ]:
#pyro.clear_param_store()
train(model, data, n_steps=500, auto_guide=False)
#clear_output(wait=True)
plt.plot(model.epoch_loss)

In [ ]:
for i in pyro.get_param_store():
    print(i, pyro.param(i))

rec = model.recommend(3, n=data.n_item)
rec['support'] = data.item['support'].loc[rec.index]
rec

In [ ]:
for i in pyro.get_param_store():
    print(i, pyro.param(i))

rec = model.recommend(3, n=data.n_item)
rec['support'] = data.item['support'].loc[rec.index]
rec

In [ ]:
results = test_chap_five(model, data, max_k=10, name='SVI_MF')
clear_output(wait=True)
results['MAP'].mean(0)[0], results['MAP'].mean(0)[-1]

## Fully bayesian model

In [ ]:
from uncertain.core import VanillaRecommender, UncertainRecommender
from uncertain.implicit.base import Implicit

import numpy as np
import torch
import pyro
import pyro.poutine as poutine
import pyro.distributions as dist
import pyro.distributions.constraints as constraints
import pyro.optim as optim

from pyro.infer import SVI, Trace_ELBO, MCMC, NUTS
from tqdm.notebook import tqdm
from torch.distributions import constraints


class BPMF(Implicit, VanillaRecommender):

    def __init__(self, n_user, n_item, embedding_dim, alpha=2):

        super().__init__()
        self.n_user = n_user
        self.n_item = n_item
        self.embedding_dim = embedding_dim

        self.alpha = alpha
        self.logit = lambda x: 1 / (1 + torch.exp(-x))

    def model2(self, user, item, Y=None):

        # Hyperparams
        mu_0 = pyro.param('µ_0', torch.zeros(self.embedding_dim))
        sigma_0 = pyro.param('sigma_0', torch.eye(self.embedding_dim), 
                             constraint=constraints.positive_definite)

        # Hyper-Priors
        mu_u = pyro.sample('µ_u', dist.MultivariateNormal(mu_0, sigma_0))
        mu_i = pyro.sample('µ_i', dist.MultivariateNormal(mu_0, sigma_0))

        # Priors
        with pyro.plate('Users', self.n_user):
            p_u = pyro.sample('p_u', dist.MultivariateNormal(mu_u, sigma_0))
        with pyro.plate('Items', self.n_item):
            q_i = pyro.sample('q_i', dist.MultivariateNormal(mu_i, sigma_0))
            
        with pyro.plate('Relevance', len(user)):
            mean = (p_u[user] * q_i[item]).sum(1)
            r_ui = pyro.sample('r_ui', dist.Normal(mean, self.alpha))
            Y_ui = pyro.sample('Y_ui', dist.Bernoulli(self.logit(r_ui)), obs=torch.tensor(Y).float())
        print(mu_u, p_u, mean, r_ui, self.logit(r_ui))
        plt.hist(r_ui.detach().numpy())

    def model(self, user, item, Y=None):

        # Hyperparams
        eta_0 = pyro.param('η_0', torch.tensor(1), constraint=constraints.positive)
        mu_0 = pyro.param('µ_0', torch.zeros(self.embedding_dim))

        # Hyper-Priors
        Lambda_u = pyro.sample('Λ_u', dist.LKJCholesky(self.embedding_dim, eta_0))
        Lambda_i = pyro.sample('Λ_i', dist.LKJCholesky(self.embedding_dim, eta_0))
        mu_u = pyro.sample('µ_u', dist.MultivariateNormal(mu_0, scale_tril=Lambda_u))
        mu_i = pyro.sample('µ_i', dist.MultivariateNormal(mu_0, scale_tril=Lambda_i))

        # Priors
        with pyro.plate('Users', self.n_user):
            p_u = pyro.sample('p_u', dist.MultivariateNormal(mu_u, scale_tril=Lambda_u))
        with pyro.plate('Items', self.n_item):
            q_i = pyro.sample('q_i', dist.MultivariateNormal(mu_i, scale_tril=Lambda_i))
            
        with pyro.plate('Relevance', len(user)):
            mean = (p_u[user] * q_i[item]).sum(1)
            r_ui = pyro.sample('r_ui', dist.Normal(mean, self.alpha))
            Y_ui = pyro.sample('Y_ui', dist.Bernoulli(self.logit(r_ui)), obs=torch.tensor(Y).float())
        #print(mu_u, p_u, mean, r_ui, self.logit(r_ui))
        #plt.hist(r_ui.detach().numpy())
    
    def guide(self, user, item, Y=None):
        
        # Variational params
        mu_u = pyro.param('µ_u_hat', torch.zeros(self.n_user, self.embedding_dim))
        mu_i = pyro.param('µ_i_hat', torch.zeros(self.n_item, self.embedding_dim))
        p_u = pyro.sample('p_u', dist.Delta(mu_u))
        q_i = pyro.sample('q_i', dist.Delta(mu_i))

        # with pyro.plate('Users', self.n_user):
           
        # with pyro.plate('Items', self.n_item):
            
        with pyro.plate('Relevance', len(user)):
            mean = (p_u[user] * q_i[item]).sum(1)
            r_ui = pyro.sample('r_ui', dist.Normal(mean, self.alpha))
            
    def do_inference(self, data, n_negative=2, n_steps=10, learning_rate=.005, auto_guide=False):

        if not hasattr(self, 'losses'):
            self.epoch_loss = []

        self.optimizer = optim.Adam({'lr': learning_rate})
        if not auto_guide:
            guide = self.guide
        else:
            guide = pyro.infer.autoguide.AutoNormal(self.model)
            # guide = pyro.infer.autoguide.AutoDelta(self.model)
        svi = SVI(self.model, guide, self.optimizer, loss=Trace_ELBO())
        for _ in (pbar := tqdm(range(n_steps), desc='Training progress')):
            
            epoch_loss = 0
            n_bat = 0
            for databat in (pbar2 := tqdm(data.train_dataloader(), leave=False, desc='Epoch progress')):
                n_bat += 1
                n = len(databat)
                user = databat[:, 0]
                item = databat[:, 1]
                
                # Sample negatives
                neg_user = np.random.randint(low=0, high=self.n_user, size=n*n_negative)
                neg_item = np.random.randint(low=0, high=self.n_item, size=n*n_negative)
                users = np.concatenate((user, neg_user))
                items = np.concatenate((item, neg_item))
                Y = np.concatenate((np.ones_like(user), np.zeros_like(neg_user)))
                
                # Update
                epoch_loss += svi.step(users, items, Y)

            self.epoch_loss.append(epoch_loss / n_bat)
            pbar.set_postfix({'ELBO loss': self.epoch_loss[-1]})

        if not auto_guide:
            self.user_embeddings = pyro.param('µ_u_hat').detach()
            self.item_embeddings = pyro.param('µ_i_hat').detach()
        else:
            self.user_embeddings = pyro.param('AutoNormal.locs.p_u').detach()
            self.item_embeddings = pyro.param('AutoNormal.locs.q_i').detach()

    def get_user_embeddings(self, user_ids):
        return self.user_embeddings[user_ids]
    
    def get_item_embeddings(self, item_ids=None):
        if item_ids is not None:
            return self.item_embeddings[item_ids]
        else:
            return self.item_embeddings
    
    def interact(self, user_embeddings, item_embeddings):
        relevance = self.logit((user_embeddings * item_embeddings).sum(1))
        return relevance
                



pyro.clear_param_store()
model = BPMF(data.n_user, data.n_item, 2, alpha=2)

display(pyro.render_model(model.model, model_args=(data.train[:, 0], 
                                                   data.train[:, 1], 
                                                   np.zeros_like(data.train[:, 0])), 
                          render_params=True, render_distributions=True))

# display(pyro.render_model(pyro.infer.autoguide.AutoDelta(model.model), model_args=(data.train[:, 0], 
#                                                    data.train[:, 1], 
#                                                    np.ones_like(data.train[:, 0])), 
#                           render_params=True, render_distributions=True))

# trace = poutine.trace(pyro.infer.autoguide.AutoDelta(model.model)).get_trace(data.train[:, 0], 
#                                              data.train[:, 1], 
#                                              np.ones_like(data.train[:, 0]))
print(trace.format_shapes())

In [ ]:
pyro.clear_param_store()
model.do_inference(data, n_steps=50, learning_rate=0.0001, auto_guide=True)
clear_output(wait=True)
plt.plot(model.epoch_loss)

In [ ]:
for i in pyro.get_param_store():
    print(i)

rec = model.recommend(3, n=data.n_item)
rec['support'] = data.item['support'].loc[rec.index]
rec

In [ ]:
model.item_embeddings

In [ ]:
results = test_vanilla(model, data, max_k=10, name='test_pyro')
clear_output(wait=True)
results['MAP']

In [ ]:
from uncertain.core import VanillaRecommender, UncertainRecommender
from uncertain.implicit.base import Implicit

import numpy as np
import torch
import pyro
import pyro.poutine as poutine
import pyro.distributions as dist
import pyro.distributions.constraints as constraints
import pyro.optim as optim

from pyro.infer import SVI, Trace_ELBO
from tqdm.notebook import tqdm


class CPMF_like(Implicit, UncertainRecommender):

    def __init__(self, n_user, n_item, embedding_dim, alpha=0.1, alpha_u=0.1, alpha_i=0.1):

        super().__init__()
        self.n_user = n_user
        self.n_item = n_item
        self.embedding_dim = embedding_dim

        self.alpha = alpha
        self.alpha_u = alpha_u
        self.alpha_i = alpha_i

        self.logit = lambda x: 1 / (1 + torch.exp(-x))

    def model(self, user, item, Y=None):
        # Register params
        mean_u = pyro.param('µ_u', torch.zeros(self.n_user, self.embedding_dim))
        mean_i = pyro.param('µ_i', torch.zeros(self.n_item, self.embedding_dim))
        gamma_u = pyro.param('γ_u', torch.ones(self.n_user))
        gamma_i = pyro.param('γ_i', torch.ones(self.n_item))
        
        # Priors
        p_u = pyro.sample('p_u', dist.Normal(mean_u, self.alpha_u).to_event(2))
        q_i = pyro.sample('q_i', dist.Normal(mean_i, self.alpha_i).to_event(2))

        with pyro.plate('relevance plate'):
            mean = (p_u[user] * q_i[item]).sum(1)
            var = gamma_u[user] * gamma_i[item] * self.alpha
            r_ui = pyro.sample('r_ui', dist.Normal(mean, var))
            Y_ui = pyro.sample('Y_ui', dist.Bernoulli(self.logit(r_ui)), obs=torch.tensor(Y).float())
        
    def guide(self, user, item, Y):
        # Register params
        mean_u = pyro.param('µ_u', torch.normal(0, 0.1, size=(self.n_user, self.embedding_dim)))
        mean_i = pyro.param('µ_i', torch.normal(0, 0.1, size=(self.n_item, self.embedding_dim)))
        gamma_u = pyro.param('γ_u', torch.ones(self.n_user))
        gamma_i = pyro.param('γ_i', torch.ones(self.n_item))
        
        # Priors
        p_u = pyro.sample('p_u', dist.Normal(mean_u, self.alpha_u).to_event(2))
        q_i = pyro.sample('q_i', dist.Normal(mean_i, self.alpha_i).to_event(2))

        with pyro.plate('relevance plate'):
            mean = (p_u[user] * q_i[item]).sum(1)
            var = torch.exp(gamma_u[user] * gamma_i[item]) * self.alpha
            r_ui = pyro.sample('r_ui', dist.Normal(mean, var))
            # Y_ui = pyro.sample('Y_ui', dist.Bernoulli(self.logit(r_ui)), infer={'is_auxiliary': True})

    def do_inference(self, data, n_negative=2, n_steps=10, learning_rate=.005):

        if not hasattr(self, 'losses'):
            self.loss = []

        self.optimizer = optim.Adam({'lr': learning_rate})
        svi = SVI(self.model, pyro.infer.autoguide.AutoNormal(), self.optimizer, loss=Trace_ELBO())
        for _ in (pbar := tqdm(range(n_steps), desc='Training progress')):
            # Print user embeddings for sanity check
            try:
                print(pyro.param('γ_u'))
            except:
                pass
                
            for databat in (pbar2 := tqdm(data.train_dataloader(), leave=False, desc='Epoch progress')):
                n = len(databat)
                user = databat[:, 0]
                item = databat[:, 1]
                
                # Sample negatives
                neg_user = np.random.randint(low=0, high=self.n_user, size=n*n_negative)
                neg_item = np.random.randint(low=0, high=self.n_item, size=n*n_negative)
                users = np.concatenate((user, neg_user))
                items = np.concatenate((item, neg_item))
                Y = np.concatenate((np.ones_like(user), np.zeros_like(neg_user)))
                
                # Update
                self.loss.append(svi.step(users, items, Y))
                pbar.set_postfix({'ELBO loss': self.loss[-1]})

        '''# IF CUSTOM GUIDE
        self.user_embeddings = pyro.param('µ_u').detach()
        self.item_embeddings = pyro.param('µ_i').detach()
        self.user_var = pyro.param('γ_u').detach()
        self.item_var = pyro.param('γ_i').detach()
        '''

        # IF AUTO GUIDE
        self.user_embeddings = pyro.param('µ_u').detach()
        self.item_embeddings = pyro.param('µ_i').detach()
        self.user_var = pyro.param('γ_u').detach()
        self.item_var = pyro.param('γ_i').detach()

    def get_user_embeddings(self, user_ids):
        return self.user_embeddings[user_ids], self.user_var[user_ids]
    
    def get_item_embeddings(self, item_ids=None):
        if item_ids is not None:
            return self.item_embeddings[item_ids], self.item_var[item_ids]
        else:
            return self.item_embeddings, self.item_var
    
    def interact(self, user_embeddings, item_embeddings):
        relevance = self.logit((user_embeddings[0] * item_embeddings[0]).sum(1))
        unc = torch.exp(user_embeddings[1] * item_embeddings[1]) * self.alpha
        return relevance, unc
                



pyro.clear_param_store()
model = CPMF_like(data.n_user, data.n_item, 16, alpha=1, alpha_u=1e-10, alpha_i=1e-10)

display(pyro.render_model(model.model, model_args=(data.train[:, 0], 
                                                   data.train[:, 1], 
                                                   np.ones_like(data.train[:, 0])), 
                          render_params=True, render_distributions=True))

trace = poutine.trace(model.model).get_trace(data.train[:, 0], 
                                             data.train[:, 1], 
                                             np.ones_like(data.train[:, 0]))
print(trace.format_shapes())

In [ ]:
pyro.clear_param_store()
model.do_inference(data, n_steps=100, learning_rate=0.001)
clear_output(wait=True)
plt.plot(model.loss)

In [ ]:
rec = model.recommend(3, n=data.n_item)
rec['support'] = data.item['support'].loc[rec.index]
rec

In [ ]:
results = test_uncertain(model, data, max_k=10, name='test_pyro')
clear_output(wait=True)
results['MAP']

In [ ]:
class BPMF(Implicit, VanillaRecommender):

    def __init__(self, n_user, n_item, embedding_dim):

        # Model dimensions
        self.n_user = n_user
        self.n_item = n_item
        self.embedding_dim = embedding_dim

        # observation noise
        self.alpha = torch.tensor(2.)

        # Init optimizer
        self.optimizer = optim.Adam({'lr': .05})

    def model(self, user, item, Y):        

        # Register parameters
        ## user
        mu0_u_prior = pyro.param('µ0_u', torch.zeros(0))
        v0_u_prior = pyro.param('v0_u', torch.tensor(self.embedding_dim))
        W0_u_prior = pyro.param('W0_u', torch.eye(self.embedding_dim))
        
        ## item
        mu0_i = pyro.param('µ0_i', self.mu0_i[item])
        v0_i = pyro.param('v0_i', self.v0_i)
        W0_i = pyro.param('W0_i', self.W0_i[item])
        
        # Sample priors
        with pyro.plate('users', self.n_user):
            Lambda_u = pyro.sample('Λ_u', dist.Wishart(self.embedding_dim, 
                                                       torch.eye(self.embedding_dim)))
            mu_u = pyro.sample('µ_u', dist.MultivariateNormal(0, 1/Lambda_u))
        with pyro.plate('items', self.n_item):
            Lambda_i = pyro.sample('Λ_i', dist.Wishart(self.embedding_dim, 
                                                       torch.eye(self.embedding_dim)))
            mu_i = pyro.sample('µ_i', dist.MultivariateNormal(0, 1/Lambda_i))

        # Sample embeddings
        with pyro.plate('user_plate', self.n_user):
            p_u = pyro.sample('user_embedding', dist.MultivariateNormal(mu_u, Lambda_u))
        with pyro.plate('item_plate', self.n_user):
            p_u = pyro.sample('item_embedding', dist.MultivariateNormal(mu_i, Lambda_i))
        
        # Sample relevance and probability of relevance
        with pyro.plate('relevance plate'):         
            r_ui = pyro.sample('r_ui', dist.Normal((p_u * q_i).sum(1), self.alpha))
            Y_ui = pyro.sample('Y_ui', dist.Bernoulli(1/(1+r_ui.exp())), obs=Y)

    def guide(self, user, item, Y):

        # Register parameters
        ## user
        mu0_u = pyro.param('µ0_u', self.mu0_u[user])
        v0_u = pyro.param('v0_u', self.v0_u)
        W0_u = pyro.param('W0_u', self.W0_u[user])
        
        ## item
        mu0_i = pyro.param('µ0_i', self.mu0_i[item])
        v0_i = pyro.param('v0_i', self.v0_i)
        W0_i = pyro.param('W0_i', self.W0_i[item])
    
        # Sample priors
        Lambda_u = pyro.sample('Λ_u', dist.Wishart(df=v0_u, covariance_matrix=torch.diag(W0_u)))
        Lambda_i = pyro.sample('Λ_i', dist.Wishart(df=v0_i, covariance_matrix=torch.diag(W0_i)))
        mu_u = pyro.sample('µ_u', dist.Normal(mu0_u, 1/Lambda_u))
        mu_i = pyro.sample('µ_i', dist.Normal(mu0_i, 1/Lambda_i))

        # Sample embeddings
        with pyro.plate('user_plate', self.n_user):
            p_u = pyro.sample('user_embedding', dist.MultivariateNormal(mu_u, Lambda_u))
        with pyro.plate('item_plate', self.n_user):
            p_u = pyro.sample('item_embedding', dist.MultivariateNormal(mu_i, Lambda_i))
        
        # Sample relevance and probability of relevance
        with pyro.plate('relevance plate'):         
            r_ui = pyro.sample('r_ui', dist.Normal((p_u * q_i).sum(1), self.alpha))

    def do_inference(self, data, n_steps=10):

        svi = SVI(self.model, self.guide, self.optimizer, loss=Trace_ELBO())
        for _ in (pbar := tqdm(range(n_steps))):
            print(self.user_var)
            for databat in (pbar2 := tqdm(data.train_dataloader(), leave=False)):
                n = len(databat)
                user = databat[:, 0]
                item = databat[:, 1]
                # Sample negatives
                neg_user = np.random.randint(low=0, high=self.n_user, size=n*2)
                neg_item = np.random.randint(low=0, high=self.n_item, size=n*2)
                users = np.concatenate((user, neg_user))
                items = np.concatenate((item, neg_item))
                Y = np.concatenate((np.ones_like(user), np.zeros_like(neg_user)))
                # Update
                loss = svi.step(users, items, Y)
                pbar.set_postfix({'loss': loss})


pyro.clear_param_store()
model = BPMF(data.n_user, data.n_item, 128)

guide = 

display(pyro.render_model(model.model, model_args=(data.train[:, 0], 
                                                    data.train[:, 1], 
                                                    np.ones_like(data.train[:, 0])), render_params=True))

trace = poutine.trace(model.model).get_trace(data.train[:, 0], data.train[:, 1], np.ones_like(data.train[:, 0]))
print(trace.format_shapes())

pyro.clear_param_store()
model.do_inference(data)